In [1]:
!pip install git+https://github.com/huggingface/accelerate
!pip install transformers==4.28.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/huggingface/accelerate to /tmp/pip-req-build-vfy1zlp2
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/accelerate /tmp/pip-req-build-vfy1zlp2
  Resolved https://github.com/huggingface/accelerate to commit bfa74e51d2af08221f5787d281d681ca9bceddd2
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for accelerate: filename=accelerate-0.20.0.dev0-py3-none-any.whl size=225536 sha256=8f43ca370c4d197949306653825586e30ba05e547a838edadadef87624d9e915
  Stored in directory: /tmp/pip-ephem-wheel-cache-awbv_rnc/wheels/f6/c7/9d/1b8a5ca8353d9307733bc719107acb67acdc95063bba749f26
Successfully built accelerate
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
!pip install git+https://github.com/huggingface/datasets -q

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 23.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 49.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 11.6 MB/s eta 0:00:00


In [3]:
from datasets import load_dataset
import torchaudio
from torchaudio import transforms
import numpy as np
from torch.utils.data import Dataset, DataLoader
import torch

In [4]:
# new implementation
import torch
import librosa
from transformers import Wav2Vec2ForCTC, Wav2Vec2Tokenizer
from datasets import load_dataset
import torchaudio
from torchaudio import transforms
import numpy as np
from torch.utils.data import Dataset, DataLoader
import torch

In [5]:
dataset = load_dataset("MLCommons/ml_spoken_words", data_dir="test", cache_dir="test", languages=["lv"])

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset ml_spoken_words downloaded and prepared to /content/test/MLCommons___ml_spoken_words/lv_wav-08367b41ae8e928f/1.0.0/321ea853cf0a05abb7a2d7efea900692a3d8622af65a2f3ce98adb7800a5d57b. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [6]:
dataset = dataset.remove_columns(["is_valid", "language", "speaker_id", "keyword", "file"])

In [7]:
# def filter_gender(example):
#     return example['gender'] in ['male', 'female']

dataset = dataset.filter(lambda features: features['gender'] in [0, 1])

Filter:   0%|          | 0/11046 [00:00<?, ? examples/s]

Filter:   0%|          | 0/1953 [00:00<?, ? examples/s]

Filter:   0%|          | 0/1874 [00:00<?, ? examples/s]

In [8]:
dataset['train'].features['gender']

ClassLabel(names=['MALE', 'FEMALE', 'OTHER', 'NAN'], id=None)

In [9]:
# labels = dataset["train"].features["gender"].names
labels = ['MALE', 'FEMALE']
label2id, id2label = dict(), dict()
for i, label in enumerate(labels):
    label2id[label] = str(i)
    id2label[str(i)] = label

In [10]:
from transformers import AutoFeatureExtractor
feature_extractor = AutoFeatureExtractor.from_pretrained("facebook/wav2vec2-base")

/usr/local/lib/python3.10/dist-packages/transformers/configuration_utils.py:379: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


In [11]:
def preprocess_function(examples):
    audio_arrays = [x["array"] for x in examples["audio"]]
    inputs = feature_extractor(
        audio_arrays, sampling_rate=feature_extractor.sampling_rate, max_length=16000, truncation=True
    )
    return inputs

In [12]:
encoded_dataset = dataset.map(preprocess_function, remove_columns="audio", batched=True)
encoded_dataset = encoded_dataset.rename_column("gender", "label")

Map:   0%|          | 0/9531 [00:00<?, ? examples/s]

Map:   0%|          | 0/1314 [00:00<?, ? examples/s]

Map:   0%|          | 0/1266 [00:00<?, ? examples/s]

In [13]:
!pip install evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 8.7 MB/s eta 0:00:00


In [14]:
# evaluation
import evaluate
accuracy = evaluate.load("accuracy")

In [15]:
import numpy as np
# def compute_metrics(eval_pred):
#     predictions = np.argmax(eval_pred.predictions, axis=1)
#     return accuracy.compute(predictions=predictions, references=eval_pred.label_ids)
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def compute_metrics(eval_pred):
    predictions = np.argmax(eval_pred.predictions, axis=1)
    precision, recall, f1, _ = precision_recall_fscore_support(eval_pred.label_ids, predictions, average=None)
    precision_macro, recall_macro, f1_macro, _ = precision_recall_fscore_support(eval_pred.label_ids, predictions, average='macro')
    acc = accuracy_score(eval_pred.label_ids, predictions)
    results = {
        'accuracy': acc,
        'precision': precision.tolist(),
        'recall': recall.tolist(),
        'f1': f1.tolist(),
        'precision_macro': precision_macro,
        'recall_macro': recall_macro,
        'f1_macro': f1_macro,
    }
    return results

In [22]:
from transformers import AutoModelForAudioClassification, TrainingArguments, Trainer
num_labels = len(id2label)
model = AutoModelForAudioClassification.from_pretrained(
    "facebook/wav2vec2-base", num_labels=num_labels, label2id=label2id, id2label=id2label
)

Some weights of the model checkpoint at facebook/wav2vec2-base were not used when initializing Wav2Vec2ForSequenceClassification: ['quantizer.codevectors', 'project_q.weight', 'project_hid.bias', 'project_hid.weight', 'quantizer.weight_proj.weight', 'project_q.bias', 'quantizer.weight_proj.bias']
- This IS expected if you are initializing Wav2Vec2ForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForSequenceClassification were not initialized from the model checkpoint at facebook/wav2vec2-base and are newly initialized: ['classifier.weight', 'projector.bias', 'projector.

In [21]:
print(torch.cuda.is_available())
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

True
cuda:0


In [23]:
training_args = TrainingArguments(
    output_dir="model",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=3e-5,
    per_device_train_batch_size=32,
    gradient_accumulation_steps=4,
    per_device_eval_batch_size=32,
    num_train_epochs=10,
    warmup_ratio=0.1,
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    push_to_hub=False,
    gradient_checkpointing=True,
)

trainer = Trainer(
    model=model.to(device),
    args=training_args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["validation"],
    tokenizer=feature_extractor,
    compute_metrics=compute_metrics,
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1,Precision Macro,Recall Macro,F1 Macro
0,0.250500,0.328427,0.929985,"[0.9159751037344398, 0.9685714285714285]","[0.9876957494407159, 0.8071428571428572]","[0.9504843918191603, 0.8805194805194805]",0.942273,0.897419,0.915502
2,0.093900,0.164406,0.942922,"[0.941747572815534, 0.9457364341085271]","[0.9765100671140939, 0.8714285714285714]","[0.958813838550247, 0.9070631970260224]",0.943742,0.923969,0.932939
2,0.094200,0.153771,0.962709,"[0.9668508287292817, 0.9535452322738386]","[0.9787472035794184, 0.9285714285714286]","[0.9727626459143969, 0.9408926417370326]",0.960198,0.953659,0.956828
4,0.144600,0.128978,0.960426,"[0.9606126914660832, 0.96]","[0.9821029082774049, 0.9142857142857143]","[0.9712389380530975, 0.9365853658536586]",0.960306,0.948194,0.953912
4,0.067200,0.129932,0.965753,"[0.9721913236929922, 0.9518072289156626]","[0.9776286353467561, 0.9404761904761905]","[0.9749023982152817, 0.9461077844311376]",0.961999,0.959052,0.960505
6,0.057800,0.125108,0.967275,"[0.9775533108866442, 0.9456264775413712]","[0.9742729306487695, 0.9523809523809523]","[0.9759103641456582, 0.9489916963226571]",0.961590,0.963327,0.962451
6,0.071200,0.126986,0.965753,"[0.975363941769317, 0.9453681710213777]","[0.9742729306487695, 0.9476190476190476]","[0.9748181309457191, 0.9464922711058265]",0.960366,0.960946,0.960655
8,0.061700,0.119879,0.968798,"[0.976536312849162, 0.9522673031026253]","[0.9776286353467561, 0.95]","[0.9770821688093907, 0.9511323003575685]",0.964402,0.963814,0.964107
8,0.037400,0.118409,0.968037,"[0.9775784753363229, 0.9478672985781991]","[0.9753914988814317, 0.9523809523809523]","[0.9764837625979843, 0.9501187648456056]",0.962723,0.963886,0.963301
9,0.040100,0.116611,0.969559,"[0.9797752808988764, 0.9481132075471698]","[0.9753914988814317, 0.9571428571428572]","[0.9775784753363229, 0.9526066350710901]",0.963944,0.966267,0.965093


Trainer is attempting to log a value of "[0.9159751037344398, 0.9685714285714285]" of type <class 'list'> for key "eval/precision" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "[0.9876957494407159, 0.8071428571428572]" of type <class 'list'> for key "eval/recall" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "[0.9504843918191603, 0.8805194805194805]" of type <class 'list'> for key "eval/f1" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "[0.941747572815534, 0.9457364341085271]" of type <class 'list'> for key "eval/precision" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "[0.9765100

TrainOutput(global_step=740, training_loss=0.10942382514476776, metrics={'train_runtime': 2696.2247, 'train_samples_per_second': 35.349, 'train_steps_per_second': 0.274, 'total_flos': 8.59519545192e+17, 'train_loss': 0.10942382514476776, 'epoch': 9.93})

In [24]:
trainer.evaluate(encoded_dataset["test"])

Trainer is attempting to log a value of "[0.9846335697399528, 0.9809523809523809]" of type <class 'list'> for key "eval/precision" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "[0.990487514863258, 0.9694117647058823]" of type <class 'list'> for key "eval/recall" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "[0.9875518672199171, 0.9751479289940828]" of type <class 'list'> for key "eval/f1" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.


{'eval_loss': 0.07553895562887192,
 'eval_accuracy': 0.9834123222748815,
 'eval_precision': [0.9846335697399528, 0.9809523809523809],
 'eval_recall': [0.990487514863258, 0.9694117647058823],
 'eval_f1': [0.9875518672199171, 0.9751479289940828],
 'eval_precision_macro': 0.9827929753461668,
 'eval_recall_macro': 0.9799496397845702,
 'eval_f1_macro': 0.9813498981069999,
 'eval_runtime': 15.2608,
 'eval_samples_per_second': 82.958,
 'eval_steps_per_second': 2.621,
 'epoch': 9.93}